In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset  # instead of load_dataset
from transformers import DataCollatorWithPadding

# Model and tokenizer
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="flex_attention",
    device_map="auto"
).to("cuda")
#model.config.sliding_window = None
model.config.use_cache = False

# Step 1: Load Excel file
df = pd.read_excel("/content/output_with_token_counts.xlsx")  # Update with your actual file path

# Optional: Check required columns
assert {"question", "solution", "answer"}.issubset(df.columns), "Excel file must contain 'question', 'solution', 'answer' columns"

# Step 2: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Optional: Split into train/validation
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Step 3: Tokenization
from transformers import DataCollatorWithPadding

# Step 3: Tokenization
tokenizer.pad_token = tokenizer.eos_token  # ensure pad_token_id is set
tokenizer.padding_side = "right"

MAX_LEN = 2048

def tokenize_for_chain(examples):
    input_id_batches = []
    label_batches = []

    for q, sol, ans in zip(examples["question"],
                           examples["solution"],
                           examples["answer"]):
        prompt = f"Question: {q}\nReasoning:"
        target = f" {sol} Answer: {ans}"

        # Tokenize prompt with special tokens
        p_ids = tokenizer.encode(prompt, add_special_tokens=True)
        # Tokenize target without special tokens
        t_ids = tokenizer.encode(target, add_special_tokens=False)

        # How many tokens we can allocate to t_ids + eos
        avail_len = MAX_LEN - len(p_ids)
        if avail_len <= 0:
            # prompt alone is too long—truncate the prompt
            p_ids = p_ids[-MAX_LEN:]
            input_ids = p_ids
            labels = [-100] * len(p_ids)
        else:
            # we need at least 1 slot for eos
            t_ids = t_ids[: avail_len - 1]
            # Concat prompt + truncated target + eos
            input_ids = p_ids + t_ids + [tokenizer.eos_token_id]
            # Mask prompt, keep target+eos as labels
            labels   = [-100] * len(p_ids) + t_ids + [tokenizer.eos_token_id]

        input_id_batches.append(input_ids)
        label_batches.append(labels)

    return {
        "input_ids": input_id_batches,
        "labels":   label_batches
    }







# use a collator that respects padding and returns tensors
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


#tokenized_train = train_dataset.map(tokenize_function, batched=True)
#tokenized_train = train_dataset.map(tokenize_function, batched=True)
#tokenized_train = tokenized_train.remove_columns(["question", "solution", "answer", "__index__"])
# After you map with tokenize_function, drop the three text columns:
# Now map it over your datasets:
tokenized_train = train_dataset.map(
    tokenize_for_chain,
    batched=True,
    remove_columns=["question","solution","answer"],
    load_from_cache_file=False,     # <<< disable the cache
)
tokenized_eval = eval_dataset.map(
    tokenize_for_chain,
    batched=True,
    remove_columns=["question","solution","answer"],
    load_from_cache_file=False,
)



#tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
from transformers import DataCollatorWithPadding
import torch

# 1) Base collator just pads inputs+labels with pad_token_id
base_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
    return_tensors="pt"
)

# 2) Wrap it to fix the labels post-pad
def data_collator(batch):
    batch = base_collator(batch)
    pad = tokenizer.pad_token_id
    # wherever labels == pad, set to -100
    batch["labels"] = torch.where(
        batch["labels"] == pad,
        torch.full_like(batch["labels"], -100),
        batch["labels"],
    )
    return batch
from transformers import TrainerCallback
training_args = TrainingArguments(
    output_dir="./qwen2.5_finetuned_limo",
    overwrite_output_dir=True,
    num_train_epochs=15,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    #learning_rate=5.0e-6,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.0,
    warmup_steps=10,
    logging_steps=1,
    save_strategy="epoch",
    ddp_timeout=180000000,
    bf16=True,
    fp16=False,
    push_to_hub=False,
    remove_unused_columns=False,   # ← add it here
    eval_strategy="no",
   # eval_steps=10
)
class DebugMetricsCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        print(f"\n>>> Step {state.global_step} metrics:", logs)

class InputLoggingCallback(TrainerCallback):
    def __init__(self):
        self.train_iter = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.train_iter = iter(kwargs["train_dataloader"])

    def on_step_begin(self, args, state, control, **kwargs):
        if state.global_step < 5:
            batch = next(self.train_iter)
            print(f"\nStep {state.global_step}:")
            print("Input IDs Shape:", batch["input_ids"].shape)
            print("Labels Shape:", batch["labels"].shape)
            print("Decoded Input:", tokenizer.decode(batch["input_ids"][0].tolist()))
            print("Decoded Labels (ignoring -100):", tokenizer.decode([x for x in batch["labels"][0].tolist() if x != -100]))

# Add callback to trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
   # callbacks=[InputLoggingCallback()]
    callbacks=[DebugMetricsCallback()],
)

#trainer.train()
# Disable torch.compile explicitly to avoid potential compilation issues with flex_attention and specific data shapes
torch._dynamo.config.disable = True
trainer.train()
# Save final model and tokenizer
model.save_pretrained("./qwen2.5_finetuned_limo")
tokenizer.save_pretrained("./qwen2.5_finetuned_limo")

Map:   0%|          | 0/735 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-20-b666e5426984>:182: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,1.379900
2,0.949100
3,1.026500
4,1.451100
5,1.110200
6,0.993700
7,1.213700
8,1.624200
9,1.187800
10,0.892500



>>> Step 1 metrics: {'loss': 1.3799, 'grad_norm': 10.375, 'learning_rate': 0.0, 'epoch': 0.0013605442176870747}

>>> Step 2 metrics: {'loss': 0.9491, 'grad_norm': 9.8125, 'learning_rate': 5e-06, 'epoch': 0.0027210884353741495}

>>> Step 3 metrics: {'loss': 1.0265, 'grad_norm': 10.6875, 'learning_rate': 1e-05, 'epoch': 0.004081632653061225}

>>> Step 4 metrics: {'loss': 1.4511, 'grad_norm': 9.875, 'learning_rate': 1.5e-05, 'epoch': 0.005442176870748299}

>>> Step 5 metrics: {'loss': 1.1102, 'grad_norm': 9.375, 'learning_rate': 2e-05, 'epoch': 0.006802721088435374}

>>> Step 6 metrics: {'loss': 0.9937, 'grad_norm': 10.3125, 'learning_rate': 2.5e-05, 'epoch': 0.00816326530612245}

>>> Step 7 metrics: {'loss': 1.2137, 'grad_norm': 9.75, 'learning_rate': 3e-05, 'epoch': 0.009523809523809525}

>>> Step 8 metrics: {'loss': 1.6242, 'grad_norm': 10.375, 'learning_rate': 3.5e-05, 'epoch': 0.010884353741496598}

>>> Step 9 metrics: {'loss': 1.1878, 'grad_norm': 10.3125, 'learning_rate': 4e-05, '

Step,Training Loss
1,1.379900
2,0.949100
3,1.026500
4,1.451100
5,1.110200
6,0.993700
7,1.213700
8,1.624200
9,1.187800
10,0.892500



>>> Step 780 metrics: {'loss': 1.165, 'grad_norm': 7.8125, 'learning_rate': 4.9401103485102435e-05, 'epoch': 1.0612244897959184}

>>> Step 781 metrics: {'loss': 0.9499, 'grad_norm': 6.53125, 'learning_rate': 4.939955114338753e-05, 'epoch': 1.0625850340136054}

>>> Step 782 metrics: {'loss': 1.0415, 'grad_norm': 9.4375, 'learning_rate': 4.939799681689171e-05, 'epoch': 1.0639455782312925}

>>> Step 783 metrics: {'loss': 0.4725, 'grad_norm': 4.875, 'learning_rate': 4.93964405057414e-05, 'epoch': 1.0653061224489795}

>>> Step 784 metrics: {'loss': 0.7234, 'grad_norm': 5.96875, 'learning_rate': 4.939488221006321e-05, 'epoch': 1.0666666666666667}

>>> Step 785 metrics: {'loss': 0.6119, 'grad_norm': 6.1875, 'learning_rate': 4.939332192998389e-05, 'epoch': 1.0680272108843538}

>>> Step 786 metrics: {'loss': 0.8553, 'grad_norm': 6.75, 'learning_rate': 4.9391759665630364e-05, 'epoch': 1.0693877551020408}

>>> Step 787 metrics: {'loss': 0.5663, 'grad_norm': 11.8125, 'learning_rate': 4.9390195417